In [2]:
import pandas as pd
import numpy as np
from math import ceil

In [10]:
teams = pd.read_csv("2021_teams.csv")

In [11]:
numTeams = len(teams)

In [15]:
# Create the Colley matrix

colley_matrix = 2*np.diag(np.ones(numTeams))
b = np.ones(numTeams)